<a href="https://colab.research.google.com/github/VadPA/PyTorch/blob/master/Lesson_3_hw_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [87]:
import numpy as np
import torch
from torch.utils.data import Dataset
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from torch import nn
import torch.nn.functional as F
from sklearn.preprocessing import MinMaxScaler

In [88]:
class Net(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 2 * hidden_dim)
        self.fc2 = nn.Linear(2 * hidden_dim, hidden_dim)
        self.bn = nn.BatchNorm1d(hidden_dim)
        self.bn2 = nn.BatchNorm1d(2 * hidden_dim)
        self.dp = nn.Dropout(0.25)
        self.fc3 = nn.Linear(1 * hidden_dim, hidden_dim)
        self.fc4 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = x.view(x.shape[0], -1)
        x = self.fc1(x)
        x = F.leaky_relu(x)
        x = self.fc2(x)
        x = F.leaky_relu(x)
        x = self.fc3(x)
        x = F.leaky_relu(x)
        x = self.fc4(x)

        return x

In [89]:
class CustomDataset(Dataset):
    def __init__(self, is_train=True):
        super(CustomDataset, self).__init__()
        self.is_train = is_train
        self.sc = MinMaxScaler()
        self.X, self.Y = fetch_california_housing(return_X_y=True, as_frame=True)
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.Y, test_size=0.25,
                                                                                random_state=13)

    def __len__(self):
        if self.is_train:
            return len(self.X_train)
        else:
            return len(self.X_test)

    def __getitem__(self, idx):
        if self.is_train:
            x = np.array(self.X_train).astype(np.float32)
            y = np.array(self.y_train).astype(np.float32)
            x = self.sc.fit_transform(x)
            return x[idx], y[idx]
        else:
            x = np.array(self.X_test).astype(np.float32)
            y = np.array(self.y_test).astype(np.float32)
            x = self.sc.fit_transform(x)
            return x[idx], y[idx]

In [90]:
X_train = CustomDataset(is_train=True)

In [91]:
train_loader = torch.utils.data.DataLoader(dataset=X_train,
                                           batch_size=64,
                                           shuffle=True)

In [92]:
X_test = CustomDataset(is_train=False)

In [93]:
test_loader = torch.utils.data.DataLoader(dataset=X_test,
                                          batch_size=64,
                                          shuffle=True)

In [94]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [95]:
net = Net(input_dim=8, hidden_dim=48, output_dim=1).to(device)  # определить сеть
print(net)  # структура сети печати

Net(
  (fc1): Linear(in_features=8, out_features=96, bias=True)
  (fc2): Linear(in_features=96, out_features=48, bias=True)
  (bn): BatchNorm1d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dp): Dropout(p=0.25, inplace=False)
  (fc3): Linear(in_features=48, out_features=48, bias=True)
  (fc4): Linear(in_features=48, out_features=1, bias=True)
)


In [96]:
loss_func = torch.nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

In [97]:
num_epochs = 5

In [98]:
for epoch in range(num_epochs):
    net.train()
    training_loss = 0.0
    training_items = 0.0
    valid_loss = 0.0

    for i, data in enumerate(train_loader):
        inputs, labels = data[0], data[1]

        # Обнуляем градиент
        optimizer.zero_grad()
        # Делаем предсказание
        outputs = net(inputs)
        # Рассчитываем лосс-функцию
        loss = loss_func(outputs.mH[0, :], labels)
        # Делаем шаг назад по лоссу
        loss.backward()
        # Делаем шаг нашего оптимайзера
        optimizer.step()

        # выводим статистику о процессе обучения
        training_loss += loss.item()
        training_items += len(labels)
        # печатаем каждые 20 mini-batches
        if i % 20 == 0:
            net.eval()
            print(f'Epoch [{epoch + 1}/{num_epochs}]. '
                  f'Step [{i + 1}/{len(train_loader)}]. '
                  f'Train_Loss: {training_loss / training_items:.4f}', end='. ')
            training_loss, training_items = 0.0, 0.0

            valid_loss, loss_val = 0.0, 0.0
            for j, batch in enumerate(test_loader):
              inputs, targets = batch
              output = net(inputs)
              loss_val = loss_func(output.mH[0, :], targets)
              valid_loss += loss_val.item()
            valid_loss /= len(test_loader)

            print('Valid_Loss: {:.4f}'.format(valid_loss))
            net.train()



Epoch [1/5]. Step [1/242]. Train_Loss: 0.1019. Valid_Loss: 5.0687
Epoch [1/5]. Step [21/242]. Train_Loss: 0.0743. Valid_Loss: 3.7313
Epoch [1/5]. Step [41/242]. Train_Loss: 0.0360. Valid_Loss: 1.2541
Epoch [1/5]. Step [61/242]. Train_Loss: 0.0216. Valid_Loss: 1.1938
Epoch [1/5]. Step [81/242]. Train_Loss: 0.0169. Valid_Loss: 1.1239
Epoch [1/5]. Step [101/242]. Train_Loss: 0.0155. Valid_Loss: 1.0498
Epoch [1/5]. Step [121/242]. Train_Loss: 0.0161. Valid_Loss: 0.9588
Epoch [1/5]. Step [141/242]. Train_Loss: 0.0147. Valid_Loss: 0.8641
Epoch [1/5]. Step [161/242]. Train_Loss: 0.0110. Valid_Loss: 0.7312
Epoch [1/5]. Step [181/242]. Train_Loss: 0.0110. Valid_Loss: 0.6358
Epoch [1/5]. Step [201/242]. Train_Loss: 0.0101. Valid_Loss: 0.6405
Epoch [1/5]. Step [221/242]. Train_Loss: 0.0094. Valid_Loss: 0.5968
Epoch [1/5]. Step [241/242]. Train_Loss: 0.0090. Valid_Loss: 0.5833
Epoch [2/5]. Step [1/242]. Train_Loss: 0.0072. Valid_Loss: 0.5945
Epoch [2/5]. Step [21/242]. Train_Loss: 0.0081. Valid_Lo

In [126]:
data_iter = iter(test_loader)
inputs, labels = data_iter.next()

In [127]:
net.eval()
outputs = net(inputs)

In [128]:
print(outputs.T)

tensor([[1.3228, 3.7831, 3.2730, 0.8615, 2.3998, 1.4612, 1.8514, 2.8138, 2.0612,
         1.9766, 0.9808, 1.3427, 1.9921, 5.9258, 1.4056, 1.1927, 4.7977, 1.1728,
         2.5367, 2.9239, 0.9644, 1.3007, 2.7993, 3.5379, 2.0879, 1.3161, 1.1940,
         1.9567, 1.7220, 1.7271, 3.1503, 1.2381, 1.8791, 2.6336, 2.9348, 0.7822,
         2.1177, 1.3714, 1.6466, 1.6983, 3.2469, 1.5990, 1.4803, 3.3077, 1.3878,
         0.9311, 2.2353, 0.9427, 1.7056, 1.1709, 1.5719, 3.0812, 0.7736, 2.0666,
         0.8278, 1.3092, 2.5248, 2.2183, 3.4351, 1.1788, 1.9628, 0.9758, 1.4200,
         2.2069]], grad_fn=<PermuteBackward0>)


In [129]:
print(loss_func(outputs.mH[0, :], labels))

tensor(0.2480, grad_fn=<MseLossBackward0>)


In [130]:
print(labels)

tensor([0.8780, 3.2470, 4.0000, 0.8180, 2.3870, 1.5520, 1.7950, 2.4690, 1.7180,
        1.9700, 0.9060, 1.0630, 2.0370, 5.0000, 1.5610, 0.7310, 5.0000, 1.4710,
        2.4790, 2.1950, 1.6610, 1.2500, 3.0080, 5.0000, 2.0860, 0.7050, 0.9140,
        2.7750, 1.3390, 2.0210, 3.6870, 1.7720, 1.5040, 2.2560, 3.6050, 0.5530,
        2.2920, 1.7500, 1.5130, 1.2530, 2.2110, 0.9120, 1.7190, 3.9230, 1.2340,
        0.5830, 3.6000, 0.5420, 1.8200, 1.1430, 1.3700, 3.1090, 0.7710, 1.5890,
        0.6690, 0.9140, 1.7170, 2.4730, 3.3670, 0.9630, 1.2600, 1.0700, 1.3110,
        3.2520])


In [104]:
# !pip install torchmetrics

In [105]:
from torchmetrics import R2Score

In [106]:
r2score = R2Score()

In [131]:
r2score(outputs[:,0], labels)

tensor(0.8033, grad_fn=<SqueezeBackward0>)

## Рассчитаем MSE для всего тестового набора

In [133]:
net.eval()
test_loss, loss_test = 0.0, 0.0
for j, batch in enumerate(test_loader):
  inputs, targets = batch
  output = net(inputs)
  loss_test = loss_func(output.mH[0, :], targets)
  test_loss += loss_test.item()
  
  
  
test_loss /= len(test_loader)
print('Test_Loss: {:.4f}'.format(test_loss))

Test_Loss: 0.4670


## Рассчитаем R2 SCORE для всего тестового набора   

In [180]:
output = net(torch.from_numpy(X_test[:][0]).type(torch.FloatTensor))

In [189]:
r2score(output[:,0], torch.from_numpy(X_test[:][1]).type(torch.FloatTensor))

tensor(0.6552, grad_fn=<SqueezeBackward0>)